In [ ]:
from pyspark import SparkContext

In [ ]:
# Inicializar SparkContext
sc = SparkContext("local", "SSSP")

In [4]:
# Función para inicializar el grafo con distancias iniciales
def initialize_graph(nodes, source):
    distances = [(node, float('inf')) for node in nodes]
    distances[source] = (source, 0.0)
    return sc.parallelize(distances)

# Función para propagar las distancias a los nodos vecinos
def propagate_distances(node, neighbors, distance):
    propagated_distances = [(neighbor, distance + 1) for neighbor in neighbors]
    propagated_distances.append((node, distance))  # Mantener la distancia actual del nodo
    return propagated_distances

# Función para actualizar las distancias en cada iteración
def update_distances(distances, propagated_distances):
    return distances.union(propagated_distances).reduceByKey(min)

In [ ]:
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, [1, 2]), (1, [0, 2, 3]), (2, [0, 1, 4]), (3, [1, 4, 5]), (4, [2, 3, 5]), (5, [3, 4])]

In [3]:
# Nodo fuente
source_node = 0

# Paso 1: Inicializar el grafo con distancias iniciales
distances = initialize_graph(nodes, source_node)

# Iterar el algoritmo hasta que se alcance la convergencia
while True:
    # Paso 2: Propagar las distancias a los nodos vecinos
    propagated_distances = sc.parallelize(edges).join(distances).flatMap(lambda x: propagate_distances(x[0], x[1][0], x[1][1]))

    # Paso 3: Actualizar las distancias
    new_distances = update_distances(distances, propagated_distances)

    # Comprobar convergencia
    if new_distances.subtract(distances).count() == 0:
        break

    distances = new_distances

# Resultado final: cada nodo junto con su distancia mínima desde el nodo fuente
result = distances.collect()
for node, distance in result:
    print("Node:", node, "Distance:", distance)

# Detener SparkContext
sc.stop()


23/06/24 18:37:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Node: 0 Distance: 0.0
Node: 1 Distance: 1.0
Node: 2 Distance: 1.0
Node: 3 Distance: 2.0
Node: 4 Distance: 2.0
Node: 5 Distance: 3.0
